Import MNE

In [1]:
import os
import numpy as numpy
import mne

Load .set data

In [2]:
calibration_path = "../BSPM/data/calibration.set"
calibration_unprocessed_path = "../BSPM/data/calibration_unprocessed.set"
eeg = mne.io.read_raw_eeglab(calibration_path,preload=True)
eeg_unprocessed = mne.io.read_raw_eeglab(calibration_unprocessed_path)
data, times = eeg[:, :]

Reading ../BSPM/data/calibration.fdt
Reading 0 ... 248153  =      0.000 ...   969.348 secs...
Reading ../BSPM/data/calibration_unprocessed.fdt


Plot EEG data. <br>
The plots were commented out due to the unavailability of the superimposed plots in MNE. For the report the necessary plots were generated on Matlab using EEGLab.

In [3]:
%matplotlib qt
#eeg.plot(duration=5,n_channels=27)
#eeg_unprocessed.plot(duration=5,n_channels=27)

Epoching on Cz (Reference channel) in order to print the error related peak at 285.9 ms. The epoching was done between 0.5 seconds before the event and 1 second after the events.

In [4]:
(events_from_annot,event_dict) = mne.events_from_annotations(eeg)
epochs_noError = mne.Epochs(eeg,events_from_annot,tmin=-0.5,tmax=1.0,event_id=7,preload=True) # S 4
epochs_Error = mne.Epochs(eeg,events_from_annot,tmin=-0.5,tmax=1.0,event_id=8,preload=True) # S 5
epochs_noError.pick_channels(['Cz'])
epochs_Error.pick_channels(['Cz'])


Used Annotations descriptions: ['R  1', 'R  2', 'R  3', 'S  1', 'S  2', 'S  3', 'S  4', 'S  5', 'S  7', 'S  8', 'S  9', 'empty']
190 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 190 events and 385 original time points ...
0 bad epochs dropped
110 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 110 events and 385 original time points ...
0 bad epochs dropped


<Epochs  |   110 events (all good), -0.5 - 1 sec, baseline [None, 0], ~358 kB, data loaded,
 '8': 110>

Average the error and no error events and plot the averages of those events.

In [5]:
ev_avg_noError = epochs_noError.average()
ev_avg_Error = epochs_Error.average()
#print(ev_avg_noError.data[0,:])
#print(ev_avg_noError.times[:])
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
plt.plot(ev_avg_noError.times[:],ev_avg_noError.data[0,:])
plt.plot(ev_avg_Error.times[:],ev_avg_Error.data[0,:])
plt.axvline(0.2852, label='Error peak', ls='--',c='green')
plt.legend(["No Error","Error", "Peak"])
plt.show()

Reepoch with all channels in order to generate the topographical plots. In order for the whole spectrum to be seen at the beginning and end of the peak all channels were included

In [6]:
epochs_noError = mne.Epochs(eeg,events_from_annot,tmin=-0.5,tmax=1.0,event_id=7,preload=True) # S 4
epochs_Error = mne.Epochs(eeg,events_from_annot,tmin=-0.5,tmax=1.0,event_id=8,preload=True) # S 5

190 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 190 events and 385 original time points ...
0 bad epochs dropped
110 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 110 events and 385 original time points ...
0 bad epochs dropped


Find the time points of the beginning and the end of the event related potentials. Both the error and non error events were averaged in order to generate the plots.

In [7]:
import math
import numpy as np
start_time = 210.9;  
end_time = 285.2;
start_index = math.floor(((start_time+500) * epochs_Error.info['sfreq']) * 0.001) #ms
end_index = math.floor(((end_time+500) * epochs_Error.info['sfreq']) * 0.001) #ms

#Get event time series.
epochs_Error_data = epochs_Error.get_data()
epochs_noError_data = epochs_noError.get_data()

#Get error related potential at the exact indexes of the timeframes.
error_210 = epochs_Error_data[:,:,start_index]
error_285 = epochs_Error_data[:,:,end_index]
noError_210 = epochs_noError_data[:,:,start_index]
noError_285 = epochs_noError_data[:,:,end_index]

#Mean event time indexes into only 1.
mean_error_210 = np.mean(error_210,axis=0)
mean_error_285 = np.mean(error_285,axis=0)
mean_noError_210 = np.mean(noError_210,axis=0)
mean_noError_285 = np.mean(noError_285,axis=0)

#Pick only the 10 selected channels.
#eeg_10 = eeg.pick_channels(processed_necessary_channel_list)

#Plot topoplots
fig, axs = plt.subplots(2,2)
axs[0,0].set_title("Error N2")
axs[0,1].set_title("Error P3")
axs[1,0].set_title("No error N2")
axs[1,1].set_title("No error P3")
mne.viz.plot_topomap(mean_error_210,eeg.info,axes=axs[0,0])
mne.viz.plot_topomap(mean_error_285,eeg.info,axes=axs[0,1])
mne.viz.plot_topomap(mean_noError_210,eeg.info,axes=axs[1,0])
mne.viz.plot_topomap(mean_noError_285,eeg.info,axes=axs[1,1])

(<matplotlib.image.AxesImage at 0x7fcf6be8a750>,
 <matplotlib.contour.QuadContourSet at 0x7fcf6beaeb90>)